# Scraping BodyBuilding.com for Exercises

## Step 1: Get base url and url extensions

In [1]:
import urllib

base_url = 'http://www.bodybuilding.com/exercises/list/index/selected/'
letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

## Step 2: Pull html and parse for name, targeted muscle, and rating information for each exercise

In [2]:
results = []

for letter in letters:
    html = str(urllib.urlopen(base_url+letter).read())
    exercises = html.split('class="exerciseName">')[1:]

    for exercise in exercises:
        info = exercise.split('</a>')
        name = (info[0].split('>')[-1])
        targeted = (info[1].split('>')[-1])

        try:
            rating = (info[2].split("rating"))[1].split('<')[0].split('>')[1]
        except:
            rating = None
            
        results.append((name, targeted, rating))
        print name, targeted, rating
        
        

 Ab Crunch Machine   Abdominals  8.1
 Ab Roller   Abdominals  8.9
 Adductor   Adductors  4.0
 Adductor/Groin    Adductors  3.6
 Advanced Kettlebell Windmill   Abdominals  8.5
 Air Bike   Abdominals  8.8
 Alien Squat   Quadriceps  None
 All Fours Quad Stretch   Quadriceps  4.9
 Alternate Hammer Curl   Biceps  8.5
 Alternate Heel Touchers   Abdominals  8.1
 Alternate Incline Dumbbell Curl   Biceps  8.7
 Alternate Leg Diagonal Bound   Quadriceps  6.9
 Alternating Cable Shoulder Press   Shoulders  7.8
 Alternating Deltoid Raise   Shoulders  9.1
 Alternating Floor Press   Chest  6.0
 Alternating Hang Clean   Hamstrings  5.0
 Alternating Kettlebell Press   Shoulders  8.6
 Alternating Kettlebell Row   Middle Back  8.1
 Alternating Leg Swing   Hamstrings  None
 Alternating Renegade Row   Middle Back  8.5
 Ankle Circles   Calves  7.4
 Ankle On The Knee   Glutes  4.8
 Anterior Tibialis-SMR   Calves  5.3
 Anti-Gravity Press   Shoulders  8.5
 Arm Circles   Shoulders  5.1
 Arnold Dumbbell Press   S

In [5]:
targets = [x[1] for x in results]
print set(targets)

set([' Lats ', ' Chest ', ' Quadriceps ', ' Biceps ', ' Middle Back ', ' Traps ', ' Shoulders ', '  ', ' Calves ', ' Abdominals ', ' Glutes ', ' Forearms ', ' Lower Back ', ' Triceps ', ' Hamstrings ', ' Abductors ', ' Adductors ', ' Neck '])


## Step 3: Reformat info for mongo

In [4]:
to_insert = []

for result in results:
    
    exercise = result[0].strip()
    muscle = result[1].strip()
    rating = result[2].strip() if result[2] else None
    
    to_insert.append({'exercise':exercise,'muscle':muscle,'rating':rating})

NameError: name 'results' is not defined

## Step 4: Insert into MongoDB

In [7]:
from pymongo import MongoClient

db_client = MongoClient()
db = db_client['exercise_db']

exercise_collection = db['exercises']

In [11]:
import re

exercise = 'pullups'

regex = re.compile(exercise,re.IGNORECASE)

muscle_groups = exercise_collection.find({"exercise" : regex})

In [12]:
from collections import Counter

counts = Counter([ex['muscle'].strip() for ex in muscle_groups])

if muscle_groups.count() == 0:
    
    print 'SPLITTING'
    
    exs = exercise.split()
    for idx in range(len(exs)-1):
        ex_string = ' '.join(exs[idx:idx+2])
        print ex_string
        regex = re.compile(ex_string,re.IGNORECASE)
        muscle_groups = exercise_collection.find({"exercise" : regex})
        temp_counter = Counter([ex['muscle'].strip() for ex in muscle_groups])
        counts.update(temp_counter)
            
    if len(counts) == 0:
        print 'SINGLES'

        for ex in exs:
            regex = re.compile(ex, re.IGNORECASE)
            muscle_groups = exercise_collection.find({"exercise" : regex})
            temp_counter = Counter([ex['muscle'].strip() for ex in muscle_groups])
            counts.update(temp_counter)

            

In [13]:
print counts

Counter({u'Lats': 4})
